# ESSE 2220 – Lab 5 Report
## Ultrasonic Distance Logging

**Course:** ESSE 2220  
**Lab Title:** Ultrasonic Distance Logger  
**Date Performed:** 2025-10-24  
**Date Submitted:** 2025-10-24

---
## 1. Names & Group Info
- **Group Number:** 5
- **Members:** Yathharthha Kaushal, Owen

---
## 2. Setup
### 2.1 Circuit Photo

![Circuit Photo](https://raw.githubusercontent.com/YK12321/ESSE2220-labs/main/5/circuit.jpeg)

### 2.2 Main Program

In [ ]:
import RPi.GPIO as GPIO
import time
import csv

trigPin = 16
echoPin = 18
MAX_DISTANCE = 220          # define the maximum measuring distance, unit: cm
timeOut = MAX_DISTANCE*60   # calculate timeout according to the maximum measuring distance
fileName = "distance_log.txt"
csvFileName = "distance_log.csv"

def writeToLog(sensorReading, time, iteration):
    # Use try-except to avoid program crash
    try:
        # Write to text file
        with open(fileName, "a") as f:
            if(iteration == 1):
                f.write("time_s    distance_cm\n")
            f.write(f"{time:.3f}    {sensorReading:.2f}\n")
        # Also write to CSV file
        with open(csvFileName, "a", newline='') as csvfile:
            csvwriter = csv.writer(csvfile)
            if(iteration == 1):
                csvwriter.writerow(["time_s", "distance_cm"])
            csvwriter.writerow([f"{time:.3f}", f"{sensorReading:.2f}"])
        return 1
    except:
        return 0

def pulseIn(pin,level,timeOut): # obtain pulse time of a pin under timeOut
    t0 = time.time()
    while(GPIO.input(pin) != level):
        if((time.time() - t0) > timeOut*0.000001):
            return 0
    t0 = time.time()
    while(GPIO.input(pin) == level):
        if((time.time() - t0) > timeOut*0.000001):
            return 0
    pulseTime = (time.time() - t0)*1000000
    return pulseTime
    
def getSonar():     # get the measurement results of ultrasonic module,with unit: cm
    GPIO.output(trigPin,GPIO.HIGH)      # make trigPin output 10us HIGH level 
    time.sleep(0.00001)     # 10us
    GPIO.output(trigPin,GPIO.LOW) # make trigPin output LOW level 
    pingTime = pulseIn(echoPin,GPIO.HIGH,timeOut)   # read plus time of echoPin
    distance = pingTime * 340.0 / 2.0 / 10000.0     # calculate distance with sound speed 340m/s 
    return distance
    
def setup():
    GPIO.setmode(GPIO.BOARD)      # use PHYSICAL GPIO Numbering
    GPIO.setup(trigPin, GPIO.OUT)   # set trigPin to OUTPUT mode
    GPIO.setup(echoPin, GPIO.IN)    # set echoPin to INPUT mode

def loop():
    # Start with iteration 1
    iteration = 1

    # Clear previous log files / create new ones at the start
    with open(fileName, "w") as f:
        f.write("")  # clear the log file at the start
    with open(csvFileName, "w", newline='') as csvfile:
        csvfile.write("")  # clear the CSV log file at the start

    while(True):
        distance = getSonar() # get distance
        # Add delay of 0.1 second between measurements
        time.sleep(0.1)
        # Write distance to log files
        writeToLog(distance, iteration*0.1, iteration)
        # Add iteration count to track time in log
        iteration = iteration + 1
        
if __name__ == '__main__':     # Program entrance
    print ('Program is starting...')
    setup()
    try:
        loop()
    except KeyboardInterrupt:  # Press ctrl-c to end the program.
        GPIO.cleanup()         # release GPIO resource

---
## 3. Observations
### 3.1 Terminal Distance Readings
- Summarize the live distance values you observed in the terminal.
- Note any trends, fluctuations, or anomalies during the session.
- Optional: paste a short snippet or screenshot of the terminal output for reference.

### 3.2 `distance_log.txt` Summary

[![Download TXT File](https://img.shields.io/badge/Download-distance__log.txt-blue?style=for-the-badge&logo=files)](https://raw.githubusercontent.com/YK12321/ESSE2220-labs/main/5/labProgram/distance_log.txt)

> The txt file stores the data collected by the sensor and the time, using a new line for each reading, and storing the time at the left, and the distance on the right with 4 spaces. The time interval for readings is 0.100 seconds, and the distance data is stored with two decimal points of precision.

### 3.3 `distance_log.csv` Plot

[![Download CSV File](https://img.shields.io/badge/Download-distance__log.csv-green?style=for-the-badge&logo=microsoftexcel)](https://raw.githubusercontent.com/YK12321/ESSE2220-labs/main/5/labProgram/distance_log.csv.xlsx)

![Distance vs Time Graph](https://raw.githubusercontent.com/YK12321/ESSE2220-labs/main/5/graph.png)

### 3.4 Sampling Parameters
- **Sampling Interval (s):** 0.100
- **Total Recording Duration (s):** 13.30s
- Briefly explain how you configured or measured these values.  
> We used a Ultrasonic distance sensor to measure the distance, and we used the time.sleep(0.1) to iterate with a 0.1s interval between sensor readings.

### 3.5 Data Consistency Discussion
- Compare the TXT and CSV outputs. Are they identical or slightly different? Explain why.
> The txt and csv outputs didn't vary a lot, except for the program having to manually add a set number of spaces after storing each data, and adding newlines. For the csv, however, the data was organized into rows/columns.
- Discuss how stable the readings were when the target object was stationary.
> The readings had a smaller variation when the target object was stationary.
- Describe what changed when the object moved faster or slower.
> As the target moved faster, the readings varied a bit more (noticeabloe by small "outelier" peaks before a the peak from a direction switch of the object's velocity.)

---
## 4. Analysis
### 4.1 Echo Pin Behavior
- Explain how the Echo pin indicates that the ultrasonic pulse has returned (signal timing, logic level, pulse width).
### 4.2 Distance Calculation Logic
- Describe the formula used and why the round-trip time is divided by 2.

### 4.3 Sources of Error
- List at least two potential causes of inaccurate or noisy readings.
- Include brief notes on how each error source appears in your data.

### 4.4 Error Mitigation Ideas
- Suggest hardware and/or software changes that could reduce the issues above.

### 4.5 Sampling Rate Trade-offs
- Consider the impact of using a shorter delay between readings on data quality, sensor reliability, or system load.

### 4.6 Extensions
- Propose how you could extend this setup to measure speed or detect motion. Outline any algorithm or additional hardware required.

---
## 5. Appendix & Checklist
- [ ] Circuit photo inserted and labeled
- [ ] Final commented code pasted
- [ ] Terminal observations summarized
- [ ] TXT and CSV screenshots included
- [ ] Distance vs Time graph embedded
- [ ] Sampling interval and duration documented
- [ ] Analysis questions answered

> Use this space for any additional notes, calculations, or raw data excerpts that support your findings.